# Garmin RSD → Side-Scan → Google Earth (Colab v4.0.1, Complete)
_Generated: 2025-09-05 18:36_

**What’s in this notebook**
- Garmin `.RSD` parser (spec-style varuint/zigzag)
- Grayscale cache → recolor (palettes: amber, blue, grayscale, green, ironbow)
- Tiling with **row-width padding fix** (handles sample count changes mid-recording)
- Three KML modes: **SINGLE**, **BUCKETED**, **REGIONATED** (LOD)
- GPS sanity markers, depth overlay, simple target detection
- Optional waterfall video (ffmpeg)
- ZIP export (optionally includes strips)
- **Synthetic sample `.RSD`** so you can run end-to-end without real data


In [0]:
# ===============================
# 1) Mount Google Drive (optional)
# ===============================
from google.colab import drive
import os, datetime
TRY_MOUNT_DRIVE = True  # set False to skip mounting
if TRY_MOUNT_DRIVE:
    drive.mount('/content/drive')
    OUTPUT_PARENT_DIR = '/content/drive/MyDrive/rsd_runs'
else:
    OUTPUT_PARENT_DIR = '/content/rsd_runs'
os.makedirs(OUTPUT_PARENT_DIR, exist_ok=True)

OUTPUT_RUN_NAME = f"run_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
OUT_DIR = os.path.join(OUTPUT_PARENT_DIR, OUTPUT_RUN_NAME)
os.makedirs(OUT_DIR, exist_ok=True)
print('OUT_DIR:', OUT_DIR)


## 2) Install dependencies

In [0]:
!pip -q install simplekml tqdm pillow numpy imageio imageio-ffmpeg scipy > /dev/null
print('Dependencies installed.')


## 3) Imports & helpers

In [0]:
from __future__ import annotations
import dataclasses, io, math, mmap, os, struct, zipfile, shlex, glob, csv
from typing import Iterator, Optional, Tuple, List
import numpy as np
from PIL import Image
from tqdm import tqdm
import simplekml

EARTH_R = 6371000.0

def meters_to_deg_lat(dy_m: float) -> float:
    return (dy_m / EARTH_R) * (180.0 / math.pi)
def meters_to_deg_lon(dx_m: float, lat_deg: float) -> float:
    return (dx_m / (EARTH_R * math.cos(math.radians(lat_deg)))) * (180.0 / math.pi)
def offset_latlon(lat: float, lon: float, dx_east_m: float, dy_north_m: float):
    return lat + meters_to_deg_lat(dy_north_m), lon + meters_to_deg_lon(dx_east_m, lat)

def haversine_m(lat1, lon1, lat2, lon2):
    if None in (lat1, lon1, lat2, lon2): return None
    r = EARTH_R
    dlat = math.radians(lat2-lat1); dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2)**2 + math.cos(math.radians(lat1))*math.cos(math.radians(lat2))*math.sin(dlon/2)**2
    return 2*r*math.asin(min(1.0, math.sqrt(a)))

class _CSV:
    def __init__(self, fp: io.TextIOBase, header: List[str]):
        self.fp = fp; self.fp.write(','.join(header) + '\n')
    def write_row(self, row: List[str]):
        safe = ['' if s is None else str(s).replace('"', "'").replace(',', ';') for s in row]
        self.fp.write(','.join(safe) + '\n')


## 4) RSD parser (spec-aligned)

In [0]:
def _read_varuint_from(mv: memoryview, pos: int):
    res = 0; shift = 0
    while True:
        b = mv[pos]; pos += 1
        res |= (b & 0x7F) << shift
        if not (b & 0x80): break
        shift += 7
    return res, pos
def _decode_varuint_bytes(b: bytes) -> int:
    res = 0; shift = 0
    for bb in b:
        res |= (bb & 0x7F) << shift
        if not (bb & 0x80): break
        shift += 7
    return res
def _zigzag_to_int32(u: int) -> int:
    return (u >> 1) ^ (-(u & 1))

@dataclasses.dataclass
class RSDRecord:
    offset: int
    channel_id: Optional[int]
    sequence_count: int
    data_size: int
    rec_time_ms: int
    lat_deg: Optional[float]
    lon_deg: Optional[float]
    water_temp_c: Optional[float]
    bottom_depth_m: Optional[float]
    sample_cnt: Optional[int]
    sonar_data_offset: Optional[int]
    sonar_data_size: Optional[int]

class RSDParser:
    MAGIC_REC_HDR = 0xB7E9DA86
    MAGIC_REC_TRL = 0xF98EACBC
    HEADER_AREA_END = 0x5000
    @staticmethod
    def mapunit_to_deg(x: int) -> float:
        return x * (360.0 / float(1 << 32))
    def __init__(self, path: str):
        self.path = path; self.size = os.path.getsize(path)
    def __enter__(self):
        self._f = open(self.path, 'rb'); self.mm = mmap.mmap(self._f.fileno(), 0, access=mmap.ACCESS_READ); return self
    def __exit__(self, exc_type, exc, tb):
        try: self.mm.close()
        finally: self._f.close()
    def _read_varstruct(self, pos: int):
        mv = memoryview(self.mm)
        n_fields, pos = _read_varuint_from(mv, pos)
        fields = []
        for _ in range(n_fields):
            key, pos = _read_varuint_from(mv, pos)
            flen_code = key & 7; field_no = key >> 3
            if flen_code == 7:
                vlen, pos = _read_varuint_from(mv, pos)
            else:
                vlen = flen_code
            val = bytes(mv[pos:pos+vlen]); pos += vlen
            fields.append((field_no, val))
        # trailing CRC (4 bytes) — ignore value, just advance
        crc = None
        if pos + 4 <= self.size:
            crc = struct.unpack('<I', self.mm[pos:pos+4])[0]
            pos += 4
        return fields, pos, crc
    def parse_records(self) -> Iterator[RSDRecord]:
        pos = self.HEADER_AREA_END
        with tqdm(total=max(0, self.size - pos), unit='B', unit_scale=True, desc='Parse RSD') as pbar:
            while pos + 12 <= self.size:
                rec_start = pos
                try:
                    fields, pos_after_hdr, _ = self._read_varstruct(pos)
                except Exception:
                    break
                magic = None; seq = 0; data_size = 0; rec_time_ms = 0
                for fn, val in fields:
                    if fn == 0 and len(val) == 4: magic = struct.unpack('<I', val)[0]
                    elif fn == 2 and len(val) == 4: seq = struct.unpack('<I', val)[0]
                    elif fn == 4 and len(val) == 2: data_size = struct.unpack('<H', val)[0]
                    elif fn == 5 and len(val) == 4: rec_time_ms = struct.unpack('<I', val)[0]
                if magic != self.MAGIC_REC_HDR:
                    pos = rec_start + 4; pbar.update(4); continue
                pos = pos_after_hdr
                body_start = pos
                channel_id = None; lat_deg = None; lon_deg = None
                water_temp_c = None; bottom_depth_m = None; sample_cnt = None
                sonar_data_offset = None; sonar_data_size = None
                if data_size > 0:
                    b_fields, body_end, _ = self._read_varstruct(body_start)
                    for fn, val in b_fields:
                        if fn == 0: channel_id = int(_decode_varuint_bytes(val))
                        elif fn == 1: bottom_depth_m = _zigzag_to_int32(_decode_varuint_bytes(val)) / 1000.0
                        elif fn == 7 and len(val) == 4: sample_cnt = struct.unpack('<I', val)[0]
                        elif fn == 9 and len(val) == 4: lat_deg = self.mapunit_to_deg(struct.unpack('<i', val)[0])
                        elif fn == 10 and len(val) == 4: lon_deg = self.mapunit_to_deg(struct.unpack('<i', val)[0])
                        elif fn == 11 and len(val) == 4: water_temp_c = struct.unpack('<f', val)[0]
                    used = body_end - body_start
                    sonar_data_offset = body_end
                    sonar_data_size = max(0, data_size - used)
                    pos = body_start + data_size
                else:
                    pos = body_start
                if pos + 12 > self.size: break
                tr_magic, chunk_size, _ = struct.unpack('<III', self.mm[pos:pos+12])
                if tr_magic != self.MAGIC_REC_TRL or chunk_size <= 0:
                    pos = rec_start + 4; pbar.update(4); continue
                yield RSDRecord(
                    offset=rec_start, channel_id=channel_id, sequence_count=seq,
                    data_size=data_size, rec_time_ms=rec_time_ms,
                    lat_deg=lat_deg, lon_deg=lon_deg,
                    water_temp_c=water_temp_c, bottom_depth_m=bottom_depth_m,
                    sample_cnt=sample_cnt, sonar_data_offset=sonar_data_offset,
                    sonar_data_size=sonar_data_size,
                )
                pos = rec_start + chunk_size; pbar.update(max(0, chunk_size))


## 5) Configuration + optional synthetic sample `.RSD`

In [0]:
# Rerun from a cached OUT_DIR (recolor & rebuild only):
RERUN_FROM_CACHE = False
CACHE_RUN_DIR = ''  # set to a previous run dir

# Input RSD (ignored if USE_SAMPLE=True or RERUN_FROM_CACHE=True)
RSD_PATH = ''

# Orientation
SWAP_SIDES = False
FLIP_PORT  = True
FLIP_STBD  = False

# Rendering / tiling
ROW_HEIGHT_PX = 40
WATER_COLUMN_PX = 8
SWATH_WIDTH_M = 40.0
MERGE_ROWS = 4
AUTO_DECIMATE = True
TILE_STRIDE = 2

# KML modes: SINGLE, BUCKETED, REGIONATED
KML_MODE = 'BUCKETED'
BUCKET_DEG = 0.01
MIN_LOD_PX = 64
MAX_LOD_PX = -1

# Tone & palettes
INVERT_INTENSITY = True
CLIP_LOW_PCT  = 1.0
CLIP_HIGH_PCT = 99.0
GAMMA = 1.0
MULTI_PALETTES = 'amber,blue,grayscale'

# Preview & video
MAKE_PREVIEW = True
PREVIEW_MAX_ROWS = 2000
MAKE_VIDEO = False
VIDEO_FPS = 15
VIDEO_MAX_WIDTH = 1280

# Extras
MAKE_GPS_MARKERS = True
GPS_MARKER_INTERVAL_M = 250
MAKE_DEPTH_OVERLAY = True
MAKE_TARGETS = True
TARGET_THRESHOLD = 220
TARGET_MIN_PIXELS = 30
TARGET_PING_GAP = 3
MAKE_ZIP = True
ZIP_INCLUDE_STRIPS = False

# Use a tiny synthetic .RSD for demo / GitHub CI
USE_SAMPLE = True

def _write_synthetic_rsd(path: str, n_pings=12, base_lat=43.132, base_lon=-83.432):
    # Minimal fake RSD-like structure to exercise the pipeline:
    # header area (0x5000), records with header/body/trailer, sonar bytes inside body.
    with open(path, 'wb') as f:
        f.write(b'\x00'*0x5000)  # header area
        seq = 0
        for i in range(n_pings):
            # ---- header varstruct ----
            hdr = bytearray()
            hdr += b'\x04'  # n_fields=4
            hdr += b'\x04' + struct.pack('<I', 0xB7E9DA86)        # field 0, magic
            hdr += b'\x14' + struct.pack('<I', seq)               # field 2, seq
            hdr += b'\x22' + b'\x00\x00'                         # field 4, data_size (patch later)
            hdr += b'\x2C' + struct.pack('<I', 1000*i)            # field 5, time_ms
            hdr_crc = struct.pack('<I', 0)

            # ---- body varstruct ----
            body = bytearray()
            body += b'\x07'  # n_fields=7
            body += b'\x00' + b'\x02'                             # ch id = 2
            depth_mm = 5000 + 50*i
            zz = (depth_mm<<1) ^ (depth_mm>>31)
            body += b'\x08' + bytes([zz & 0x7F])                  # depth_mm zigzag (tiny)
            body += b'\x38' + struct.pack('<I', 512)              # sample_cnt=512
            lat_mu = int((base_lat + 0.0005*i) * (1<<32) / 360.0)
            lon_mu = int((base_lon + 0.0005*i) * (1<<32) / 360.0)
            body += b'\x48' + struct.pack('<i', lat_mu)           # lat
            body += b'\x50' + struct.pack('<i', lon_mu)           # lon
            body += b'\x58' + struct.pack('<f', 18.0 + 0.1*i)     # temp
            body_crc = struct.pack('<I', 0)

            # sonar bytes (u8, 2ch) inside body region
            sonar = os.urandom(1024)
            data_size = len(body) + 4 + len(sonar)
            hdr_patched = bytearray(hdr)
            # Patch data_size value (find the two bytes after field tag 0x22)
            # hdr layout: n(1) + [f0 tag(1)+4] + [f2 tag(1)+4] + [f4 tag(1)+2] + [f5 tag(1)+4]
            # indexes:          0..5             6..11            12..14             15..20
            hdr_patched[13:15] = struct.pack('<H', data_size)

            # write: header, hdr_crc, body, body_crc, sonar, then trailer
            f.write(hdr_patched); f.write(hdr_crc)
            body_start_len = len(body) + 4
            f.write(body); f.write(body_crc)
            f.write(sonar)

            # trailer (covers header+hdr_crc + body+body_crc + sonar + trailer itself)
            chunk_size = len(hdr_patched)+4 + body_start_len + len(sonar) + 12
            f.write(struct.pack('<I', 0xF98EACBC))  # magic
            f.write(struct.pack('<I', chunk_size))
            f.write(struct.pack('<I', 0))           # crc
            seq += 1

if USE_SAMPLE:
    SAMPLE_PATH = '/content/sample.rsd'
    _write_synthetic_rsd(SAMPLE_PATH)
    RSD_PATH = SAMPLE_PATH
    print('Synthetic sample written:', RSD_PATH)
else:
    print('Upload your .RSD via the file browser or set RSD_PATH manually.')


## 6) Tone mapping & palettes

In [0]:
def make_palette(name: str) -> np.ndarray:
    name = (name or 'grayscale').lower().strip()
    x = np.linspace(0, 1, 256)
    if name == 'amber':
        r = np.clip(3.0*x, 0, 1); g = np.clip(1.8*x, 0, 1); b = np.clip(0.5*x, 0, 1)
    elif name == 'blue':
        r = np.clip(0.4*x, 0, 1); g = np.clip(0.7*x, 0, 1); b = np.clip(1.8*x, 0, 1)
    elif name == 'green':
        r = np.clip(0.5*x, 0, 1); g = np.clip(1.8*x, 0, 1); b = np.clip(0.5*x, 0, 1)
    elif name == 'ironbow':
        r = np.clip(1.5*x, 0, 1); g = np.clip(1.5*np.maximum(x-0.33,0), 0, 1); b = np.clip(1.5*np.maximum(x-0.66,0), 0, 1)
    else:
        r = g = b = x
    return (np.stack([r,g,b], axis=1) * 255.0 + 0.5).astype(np.uint8)

def tone_map(u8: np.ndarray) -> np.ndarray:
    a = u8.astype(np.float32)
    if INVERT_INTENSITY: a = 255.0 - a
    lo = np.percentile(a, CLIP_LOW_PCT) if 0 <= CLIP_LOW_PCT < 50 else a.min()
    hi = np.percentile(a, CLIP_HIGH_PCT) if 50 < CLIP_HIGH_PCT <= 100 else a.max()
    if hi <= lo: hi = lo + 1.0
    a = (a - lo) / (hi - lo)
    if GAMMA != 1.0: a = np.power(np.clip(a, 0, 1), GAMMA)
    return np.clip(a * 255.0, 0, 255).astype(np.uint8)

def _normalize_to_u8(arr: np.ndarray, is_u16: bool) -> np.ndarray:
    a = arr.astype(np.float32)
    if is_u16: a *= (255.0/65535.0)
    return np.clip(a, 0, 255).astype(np.uint8)

def _infer_layout(blob_len: int, sample_cnt: Optional[int]):
    sc = int(sample_cnt or 0)
    if sc > 0:
        if blob_len == sc:     return ('u8', 1)
        if blob_len == 2*sc:   return ('u8', 2)
        if blob_len == 4*sc:   return ('u16', 2)
        if blob_len == 2*sc:   return ('u16', 1)
        ratio = blob_len / sc
        if abs(ratio-2.0) < 0.1: return ('u8', 2)
        if abs(ratio-1.0) < 0.1: return ('u8', 1)
        if abs(ratio-4.0) < 0.2: return ('u16', 2)
        if abs(ratio-2.0) < 0.2: return ('u16', 1)
    return ('u8', 2)

def assemble_scan(a: np.ndarray, sc: int, chans: int) -> np.ndarray:
    if sc <= 0: sc = a.size // chans
    if chans == 2 and a.size >= 2*sc:
        port = a[:sc]; stbd = a[sc:2*sc]
        if FLIP_PORT: port = port[::-1]
        if FLIP_STBD: stbd = stbd[::-1]
        if SWAP_SIDES: port, stbd = stbd, port
        scan = np.hstack([port, np.zeros(WATER_COLUMN_PX, dtype=np.uint8), stbd])
    else:
        scan = a[:sc]
    return scan


## 7) Parse → CSV/track → grayscale row cache

In [0]:
def load_or_parse_rsd(path, out_dir):
    base = os.path.splitext(os.path.basename(path))[0]
    recs = []
    with RSDParser(path) as P:
        for rec in P.parse_records(): recs.append(rec)
    # CSV
    csv_path = os.path.join(out_dir, f"{base}_records.csv")
    with open(csv_path, 'w', encoding='utf-8') as fp:
        w = _CSV(fp, ['file_ofs','channel_id','seq','time_ms','lat_deg','lon_deg','water_temp_c','bottom_depth_m','sample_cnt','sonar_ofs','sonar_size'])
        for r in recs:
            w.write_row([r.offset, r.channel_id or '', r.sequence_count, r.rec_time_ms,
                         f"{r.lat_deg:.8f}" if r.lat_deg is not None else '',
                         f"{r.lon_deg:.8f}" if r.lon_deg is not None else '',
                         f"{r.water_temp_c:.2f}" if r.water_temp_c is not None else '',
                         f"{r.bottom_depth_m:.3f}" if r.bottom_depth_m is not None else '',
                         r.sample_cnt or '', r.sonar_data_offset or '', r.sonar_data_size or ''])
    # Track
    trk_kml = os.path.join(out_dir, f"{base}_track.kml")
    k = simplekml.Kml(); ls = k.newlinestring(name=f"{base} track"); coords=[]
    for r in recs:
        if r.lat_deg is None or r.lon_deg is None: continue
        alt = -float(r.bottom_depth_m) if r.bottom_depth_m is not None else 0.0
        coords.append((r.lon_deg, r.lat_deg, alt))
    ls.coords = coords; k.save(trk_kml)

    # Grayscale rows cache
    with open(path, 'rb') as f:
        preview_rows = []
        for idx, r in enumerate(recs):
            if not r.sonar_data_offset or not r.sonar_data_size or r.sonar_data_size <= 0: continue
            f.seek(r.sonar_data_offset); blob = f.read(r.sonar_data_size)
            dtype, chans = _infer_layout(len(blob), r.sample_cnt)
            if dtype == 'u8': a = np.frombuffer(blob, dtype=np.uint8)
            else: a = np.frombuffer(blob, dtype='<u2'); a = _normalize_to_u8(a, True)
            sc = int(r.sample_cnt or 0)
            gray = tone_map(assemble_scan(a, sc, chans))
            if gray.ndim == 1: gray = gray[np.newaxis, :]
            img = Image.fromarray(gray, 'L').resize((gray.shape[1], ROW_HEIGHT_PX))
            img.save(os.path.join(out_dir, f"{base}_rowgray_{idx:06d}.png"))
            if MAKE_PREVIEW and idx < PREVIEW_MAX_ROWS:
                preview_rows.append(np.array(img.convert('RGB')))
            if (idx+1) % 2000 == 0: print('rows (gray) written:', idx+1)
    if MAKE_PREVIEW and preview_rows:
        prev = np.vstack(preview_rows)
        Image.fromarray(prev, 'RGB').save(os.path.join(out_dir, f"{base}_grayscale_waterfall.png"))
    return base, recs, trk_kml


## 8) Recolor rows & build tiles (with padding fix)

In [0]:
def recolor_rows(out_dir, base, palette_name, make_preview=True):
    lut = make_palette(palette_name)
    row_files = sorted(glob.glob(os.path.join(out_dir, f"{base}_rowgray_*.png")))
    preview_rows = []
    for p in row_files:
        gray = np.array(Image.open(p))
        rgb = lut[gray]
        op = p.replace('_rowgray_', f'_{palette_name}_row_')
        Image.fromarray(rgb, 'RGB').save(op)
        if make_preview and len(preview_rows) < PREVIEW_MAX_ROWS:
            preview_rows.append(rgb)
    if make_preview and preview_rows:
        prev = np.vstack(preview_rows)
        Image.fromarray(prev, 'RGB').save(os.path.join(out_dir, f"{base}_{palette_name}_waterfall.png"))

def segment_quad(latA, lonA, latB, lonB, half_width_m):
    dN = (latB - latA) * (math.pi/180.0) * EARTH_R
    dE = (lonB - lonA) * (math.pi/180.0) * EARTH_R * math.cos(math.radians((latA+latB)/2.0))
    L  = max(1e-6, math.hypot(dE, dN))
    uE, uN = dE/L, dN/L
    pE, pN = -uN, uE
    A_left  = offset_latlon(latA, lonA,  pE*half_width_m,  pN*half_width_m)
    A_right = offset_latlon(latA, lonA, -pE*half_width_m, -pN*half_width_m)
    B_left  = offset_latlon(latB, lonB,  pE*half_width_m,  pN*half_width_m)
    B_right = offset_latlon(latB, lonB, -pE*half_width_m, -pN*half_width_m)
    return [(A_left[1],A_left[0]), (A_right[1],A_right[0]), (B_right[1],B_right[0]), (B_left[1],B_left[0])]

def _pad_row_center_to_width(row_rgb: np.ndarray, target_w: int) -> np.ndarray:
    h, w, c = row_rgb.shape
    if w == target_w:
        return row_rgb
    delta = target_w - w
    left = delta // 2
    right = delta - left
    return np.pad(row_rgb, ((0,0), (left,right), (0,0)), mode='constant', constant_values=0)

def build_tiles(out_dir, base, recs, palette_name, merge_rows, tile_stride, swath_m):
    used = 0; i = 0; tiles_meta = []
    while i < len(recs) - 1:
        valid_pairs = []; rows_for_tile = []
        j = i
        while j < min(i + merge_rows, len(recs) - 1):
            rA, rB = recs[j], recs[j+1]
            if None not in (rA.lat_deg, rA.lon_deg, rB.lat_deg, rB.lon_deg):
                fp = os.path.join(out_dir, f"{base}_{palette_name}_row_{j:06d}.png")
                if os.path.exists(fp):
                    rows_for_tile.append(np.array(Image.open(fp)))
                    valid_pairs.append((rA, rB))
            j += 1
        if rows_for_tile and valid_pairs:
            max_w = max(r.shape[1] for r in rows_for_tile)
            rows_for_tile = [_pad_row_center_to_width(r, max_w) for r in rows_for_tile]
            tile_img = np.vstack(rows_for_tile)
            tname = f"{base}_{palette_name}_tile_{i:06d}.png"
            tpath = os.path.join(out_dir, tname)
            Image.fromarray(tile_img, 'RGB').save(tpath)
            rA0, rB1 = valid_pairs[0][0], valid_pairs[-1][1]
            quad = segment_quad(rA0.lat_deg, rA0.lon_deg, rB1.lat_deg, rB1.lon_deg, swath_m)
            lons = [q[0] for q in quad]; lats = [q[1] for q in quad]
            bbox = dict(n=max(lats), s=min(lats), e=max(lons), w=min(lons))
            tiles_meta.append(dict(name=tname, path=tpath, quad=quad, bbox=bbox))
            used += 1
        i += max(1, tile_stride)
    return tiles_meta, used


## 9) KML writers (SINGLE, BUCKETED, REGIONATED)

In [0]:
def kml_region(bbox, min_px, max_px):
    return f"""
    <Region>
      <LatLonAltBox>
        <north>{bbox['n']}</north>
        <south>{bbox['s']}</south>
        <east>{bbox['e']}</east>
        <west>{bbox['w']}</west>
      </LatLonAltBox>
      <Lod>
        <minLodPixels>{int(min_px)}</minLodPixels>
        <maxLodPixels>{int(max_px)}</maxLodPixels>
      </Lod>
    </Region>
    """.strip()

def write_kmz_single(out_dir, base, palette_name, tiles_meta):
    kmz_path = os.path.join(out_dir, f"{base}_{palette_name}_sidescan.kmz")
    images_dir_in_kmz = 'files'
    kml = ['<?xml version="1.0" encoding="UTF-8"?>',
           '<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2">',
           '<Document>', f'<name>{base} {palette_name} sidescan</name>']
    for T in tiles_meta:
        coords_txt = ' '.join([f"{lon:.8f},{lat:.8f}" for lon,lat in T['quad']])
        kml += ['<GroundOverlay>', f'  <name>{T["name"]}</name>', '  <Icon>',
                f'    <href>files/' + T['name'] + '</href>', '  </Icon>',
                '  <gx:LatLonQuad>', f'    <coordinates>{coords_txt}</coordinates>', '  </gx:LatLonQuad>',
                '</GroundOverlay>']
    kml += ['</Document>', '</kml>']
    with zipfile.ZipFile(kmz_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        zf.writestr('doc.kml', '\n'.join(kml))
        for T in tiles_meta: zf.write(T['path'], f"{images_dir_in_kmz}/{T['name']}")
    return kmz_path

def assign_buckets(tiles_meta, bucket_deg):
    buckets = {}
    for T in tiles_meta:
        latc = sum([p[1] for p in T['quad']])/4.0
        lonc = sum([p[0] for p in T['quad']])/4.0
        ib = int(math.floor(latc / bucket_deg))
        jb = int(math.floor(lonc / bucket_deg))
        buckets.setdefault((ib,jb), []).append(T)
    return buckets

def write_kmz_bucketed(out_dir, base, palette_name, tiles_meta, bucket_deg):
    kmz_path = os.path.join(out_dir, f"{base}_{palette_name}_sidescan.kmz")
    images_dir_in_kmz = 'files'
    buckets = assign_buckets(tiles_meta, bucket_deg)
    master = ['<?xml version="1.0" encoding="UTF-8"?>',
              '<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2">',
              '<Document>', f'<name>{base} {palette_name} sidescan (bucketed)</name>']
    for (ib,jb), tiles in buckets.items():
        master += [f'<Folder><name>bucket {ib},{jb}</name>']
        for T in tiles:
            coords_txt = ' '.join([f"{lon:.8f},{lat:.8f}" for lon,lat in T['quad']])
            master += ['<GroundOverlay>', f'  <name>{T["name"]}</name>', '  <Icon>',
                       f'    <href>files/' + T['name'] + '</href>', '  </Icon>',
                       '  <gx:LatLonQuad>', f'    <coordinates>{coords_txt}</coordinates>', '  </gx:LatLonQuad>',
                       '</GroundOverlay>']
        master += ['</Folder>']
    master += ['</Document>', '</kml>']
    with zipfile.ZipFile(kmz_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        zf.writestr('doc.kml', '\n'.join(master))
        for T in tiles_meta: zf.write(T['path'], f"{images_dir_in_kmz}/{T['name']}")
    return kmz_path

def write_kmz_regionated(out_dir, base, palette_name, tiles_meta, bucket_deg, min_px, max_px):
    kmz_path = os.path.join(out_dir, f"{base}_{palette_name}_sidescan.kmz")
    images_dir_in_kmz = 'files'
    buckets = assign_buckets(tiles_meta, bucket_deg)
    def bucket_bbox(tiles):
        n = max(t['bbox']['n'] for t in tiles)
        s = min(t['bbox']['s'] for t in tiles)
        e = max(t['bbox']['e'] for t in tiles)
        w = min(t['bbox']['w'] for t in tiles)
        return dict(n=n,s=s,e=e,w=w)
    master = ['<?xml version="1.0" encoding="UTF-8"?>',
              '<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2">',
              '<Document>', f'<name>{base} {palette_name} sidescan (regionated)</name>']
    child_docs = []
    for key, tiles in buckets.items():
        bbox = bucket_bbox(tiles)
        child_name = f"bucket_{key[0]}_{key[1]}.kml"
        master += ['<NetworkLink>', f'  <name>{child_name}</name>', kml_region(bbox, min_px, max_px),
                   '  <Link>', f'    <href>kml/' + child_name + '</href>', '    <viewRefreshMode>onRegion</viewRefreshMode>', '  </Link>', '</NetworkLink>']
        child = ['<?xml version="1.0" encoding="UTF-8"?>',
                 '<kml xmlns="http://www.opengis.net/kml/2.2" xmlns:gx="http://www.google.com/kml/ext/2.2">',
                 '<Document>', f'<name>{child_name}</name>']
        for T in tiles:
            coords_txt = ' '.join([f"{lon:.8f},{lat:.8f}" for lon,lat in T['quad']])
            child += ['<GroundOverlay>', f'  <name>{T["name"]}</name>', kml_region(T['bbox'], min_px, max_px),
                      '  <Icon>', f'    <href>../files/' + T['name'] + '</href>', '  </Icon>',
                      '  <gx:LatLonQuad>', f'    <coordinates>{coords_txt}</coordinates>', '  </gx:LatLonQuad>', '</GroundOverlay>']
        child += ['</Document>', '</kml>']
        child_docs.append((child_name, '\n'.join(child)))
    master += ['</Document>', '</kml>']
    with zipfile.ZipFile(kmz_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        zf.writestr('doc.kml', '\n'.join(master))
        for name, text in child_docs: zf.writestr(f'kml/{name}', text)
        for T in tiles_meta: zf.write(T['path'], f"{images_dir_in_kmz}/{T['name']}")
    return kmz_path


## 10) Waterfall video export (ffmpeg)

In [0]:
def ffmpeg_video_from_rows(out_dir, base, palette_name, fps=15, max_w=1280):
    list_path = os.path.join(out_dir, f"{base}_{palette_name}_frames.txt")
    with open(list_path, 'w') as f:
        for p in sorted(glob.glob(os.path.join(out_dir, f"{base}_{palette_name}_row_*.png"))):
            f.write(f"file '{p}'\n")
    mp4_path = os.path.join(out_dir, f"{base}_{palette_name}_waterfall.mp4")
    cmd = f"ffmpeg -y -f concat -safe 0 -i {shlex.quote(list_path)} -vf \"scale=min(iw\\,{int(max_w)}):-2,fps={int(fps)}\" -c:v libx264 -pix_fmt yuv420p -preset veryfast -crf 23 {shlex.quote(mp4_path)}"
    print(cmd); os.system(cmd); print('Video:', mp4_path); return mp4_path


## 11) GPS markers (sanity checks)

In [0]:
def gps_markers_kml(out_dir, base, recs, interval_m):
    k = simplekml.Kml(); last_lat=None; last_lon=None; acc=0.0; count=0
    for r in recs:
        if r.lat_deg is None or r.lon_deg is None: continue
        if last_lat is None:
            last_lat, last_lon = r.lat_deg, r.lon_deg
            k.newpoint(name='Start', coords=[(r.lon_deg, r.lat_deg)])
            continue
        d = haversine_m(last_lat,last_lon,r.lat_deg,r.lon_deg) or 0.0
        acc += d; last_lat, last_lon = r.lat_deg, r.lon_deg
        if acc >= interval_m:
            count += 1; acc = 0.0
            k.newpoint(name=f'GP {count}', coords=[(r.lon_deg, r.lat_deg)])
    path = os.path.join(out_dir, f"{base}_gpsmarkers.kml"); k.save(path); return path


## 12) Depth overlay (CSV + KML path)

In [0]:
def depth_overlay(out_dir, base, recs):
    csv_p = os.path.join(out_dir, f"{base}_depth.csv")
    with open(csv_p, 'w', newline='', encoding='utf-8') as fp:
        w = csv.writer(fp); w.writerow(['time_ms','lat_deg','lon_deg','depth_m','temp_c'])
        for r in recs:
            w.writerow([r.rec_time_ms, r.lat_deg, r.lon_deg, r.bottom_depth_m, r.water_temp_c])
    k = simplekml.Kml(); ls = k.newlinestring(name=f"{base} depth"); coords=[]
    for r in recs:
        if r.lat_deg is None or r.lon_deg is None: continue
        coords.append((r.lon_deg, r.lat_deg))
    ls.coords = coords; ls.extrude = 0; ls.altitudemode = simplekml.AltitudeMode.clamptoground
    kml_p = os.path.join(out_dir, f"{base}_depth.kml"); k.save(kml_p)
    return csv_p, kml_p


## 13) Target detection (simple bright-blob heuristic)

In [0]:
def detect_targets(out_dir, base, recs, thresh=220, min_pixels=30, ping_gap=3):
    from scipy import ndimage as ndi
    rows = sorted(glob.glob(os.path.join(out_dir, f"{base}_rowgray_*.png")))
    targets = []
    for p in rows:
        idx = int(os.path.basename(p).split('_rowgray_')[1].split('.')[0])
        rA = recs[idx] if idx < len(recs) else None
        if rA is None or rA.lat_deg is None or rA.lon_deg is None: continue
        img = np.array(Image.open(p))
        mask = img >= int(thresh)
        if not mask.any(): continue
        lab, n = ndi.label(mask)
        if n == 0: continue
        areas = ndi.sum(mask, lab, index=np.arange(1,n+1))
        means = ndi.mean(img, lab, index=np.arange(1,n+1))
        for k_lbl in range(1, n+1):
            area = int(areas[k_lbl-1])
            if area < min_pixels: continue
            meanv = float(means[k_lbl-1])
            targets.append((idx, rA.lat_deg, rA.lon_deg, area, meanv))
    targets.sort(key=lambda x: x[0])
    merged = []
    for t in targets:
        if not merged or t[0] - merged[-1][0] > ping_gap:
            merged.append(list(t))
        else:
            if t[3]*t[4] > merged[-1][3]*merged[-1][4]: merged[-1] = list(t)
    csv_p = os.path.join(out_dir, f"{base}_targets.csv")
    with open(csv_p, 'w', newline='', encoding='utf-8') as fp:
        w = csv.writer(fp); w.writerow(['ping_idx','lat','lon','area_px','mean_u8'])
        for t in merged: w.writerow(t)
    k = simplekml.Kml()
    for i, t in enumerate(merged, 1):
        k.newpoint(name=f'Target {i}', coords=[(t[2], t[1])], description=f'ping={t[0]}, area_px={t[3]}, mean={t[4]:.1f}')
    kml_p = os.path.join(out_dir, f"{base}_targets.kml"); k.save(kml_p)
    return csv_p, kml_p, len(merged)


## 14) Orchestrate pipeline + ZIP export

In [0]:
def auto_stride(n_pings:int) -> int:
    if n_pings <= 20_000: return 1
    if n_pings <= 50_000: return 2
    if n_pings <= 100_000: return 3
    if n_pings <= 200_000: return 5
    return 8

if RERUN_FROM_CACHE:
    assert os.path.isdir(CACHE_RUN_DIR), 'Set CACHE_RUN_DIR to a prior run folder.'
    OUT_DIR = CACHE_RUN_DIR
    gray_files = sorted(glob.glob(os.path.join(OUT_DIR, '*_rowgray_*.png')))
    assert gray_files, 'No cached grayscale rows found.'
    base = os.path.basename(gray_files[0]).split('_rowgray_')[0]
    rec_csv = os.path.join(OUT_DIR, f"{base}_records.csv")
    recs = []
    with open(rec_csv, 'r', encoding='utf-8') as fp:
        hdr = fp.readline()
        for line in fp:
            cols = line.strip().split(',')
            def f(x):
                try: return float(x) if x else None
                except: return None
            recs.append(RSDRecord(0,None,0,0,0,f(cols[4]),f(cols[5]),None,f(cols[7]),None,None,None))
else:
    assert RSD_PATH and os.path.exists(RSD_PATH), 'RSD_PATH not set or file not found.'
    base, recs, trk = load_or_parse_rsd(RSD_PATH, OUT_DIR)

n_pings = len(recs)
stride = (auto_stride(n_pings) if AUTO_DECIMATE else max(1, int(TILE_STRIDE)))
print(f'Ping count: {n_pings} -> TILE_STRIDE={stride}')

palettes = [p.strip() for p in MULTI_PALETTES.split(',') if p.strip()]
kmz_paths = []
for pal in palettes:
    print('Recolor:', pal)
    recolor_rows(OUT_DIR, base, pal, make_preview=MAKE_PREVIEW)
    tiles_meta, used = build_tiles(OUT_DIR, base, recs, pal, MERGE_ROWS, stride, SWATH_WIDTH_M)
    print(f'Tiles built ({pal}):', used)
    if KML_MODE == 'SINGLE':
        kmz = write_kmz_single(OUT_DIR, base, pal, tiles_meta)
    elif KML_MODE == 'BUCKETED':
        kmz = write_kmz_bucketed(OUT_DIR, base, pal, tiles_meta, BUCKET_DEG)
    else:
        kmz = write_kmz_regionated(OUT_DIR, base, pal, tiles_meta, BUCKET_DEG, MIN_LOD_PX, MAX_LOD_PX)
    kmz_paths.append(kmz)
    if MAKE_VIDEO:
        ffmpeg_video_from_rows(OUT_DIR, base, pal, fps=VIDEO_FPS, max_w=VIDEO_MAX_WIDTH)

if MAKE_GPS_MARKERS:
    gp = gps_markers_kml(OUT_DIR, base, recs, GPS_MARKER_INTERVAL_M)
    print('GPS markers KML:', gp)
if MAKE_DEPTH_OVERLAY:
    dcsv, dkml = depth_overlay(OUT_DIR, base, recs)
    print('Depth CSV:', dcsv); print('Depth KML:', dkml)
if MAKE_TARGETS:
    tcsv, tkml, nT = detect_targets(OUT_DIR, base, recs, TARGET_THRESHOLD, TARGET_MIN_PIXELS, TARGET_PING_GAP)
    print(f'Targets: {nT} -> {tcsv}, {tkml}')

if MAKE_ZIP:
    zip_name = os.path.basename(OUT_DIR.rstrip('/')) + '.zip'
    zip_path = os.path.join(OUT_DIR, zip_name)
    with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as z:
        pats = [f"*records.csv", f"*track.kml", f"*targets.*", f"*depth.*", f"*_*_sidescan.kmz", f"*_*_waterfall.png", f"*_*_waterfall.mp4"]
        for pat in pats:
            for p in glob.glob(os.path.join(OUT_DIR, pat)):
                z.write(p, os.path.basename(p))
        if ZIP_INCLUDE_STRIPS:
            for p in glob.glob(os.path.join(OUT_DIR, f"*_*_row_*.png")):
                z.write(p, os.path.join('strips', os.path.basename(p)))
    print('ZIP:', zip_path)
    try:
        from google.colab import files as _files
        _files.download(zip_path)
    except Exception:
        pass

print('Done. OUT_DIR:', OUT_DIR)
